## Imports

In [1]:
from pandas import read_csv
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold
from sklearn.preprocessing import StandardScaler
from skopt import BayesSearchCV
# sklearn version 0.23.2 needed
from sklearn.svm import SVC
%matplotlib inline

## Data
Ionosphere data set: https://raw.githubusercontent.com/jbrownlee/Datasets/master/ionosphere.names

In [2]:
# load dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/ionosphere.csv'
dataframe = read_csv(url, header=None)
# split into input and output elements
data = dataframe.values
X, y = data[:, :-1], data[:, -1]

## Scale X 
Tends to speed up SVC

In [3]:
scaler = StandardScaler().fit(X)
scaled_X = scaler.transform(X)

## Search Space for SVC
Space suggested by https://machinelearningmastery.com/scikit-optimize-for-hyperparameter-tuning-in-machine-learning/

In [4]:
params = dict()
params['C'] = (1e-6, 100.0, 'log-uniform')
params['gamma'] = (1e-6, 100.0, 'log-uniform')
params['degree'] = (1,5)
params['kernel'] = ['linear', 'poly', 'rbf', 'sigmoid']

## Cross validation method

In [5]:
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

## Initialize optimizer

In [6]:
est = SVC(cache_size=2000, probability=True)

In [9]:
search = BayesSearchCV(estimator=est, search_spaces=params, n_jobs=6, cv=cv, n_iter=10, scoring ='neg_log_loss')
#scoring ='neg_log_loss'
#scoring = 'roc_auc'

## Run!

In [10]:
search.fit(scaled_X, y)
# Score on best result
print(search.best_score_)
print(search.best_params_)

-0.35834025620600385
OrderedDict([('C', 0.21272165134492205), ('degree', 1), ('gamma', 3.967912310113454e-05), ('kernel', 'poly')])
